In [1]:
# 用户提问：
question = '如何登录？'

In [2]:
## GPU 0
## Lib Load
import numpy as np
import heapq
from sentence_transformers import SentenceTransformer,util

m = SentenceTransformer("../text2vec/")

## question embedding
input_embeddings = m.encode(question)

No sentence-transformers model found with name ../text2vec/. Creating a new one with MEAN pooling.


In [3]:
## Lib Embeddings
from Doclib.load_Filelist import filelist
fl,fnl = filelist()

# Embeddings Extraction
embed_list = []
for _fl in fnl:
    _embeddings = m.encode(_fl)
    embed_list.append(_embeddings)

In [4]:
# Simlarity Calculate
cos_list = []
for _embeddings in embed_list:
    _score = util.pytorch_cos_sim(input_embeddings, _embeddings)
    cos_list.append(float(_score))

In [5]:
#topk
# 获取下标， 输出top 2下标
fnumb = heapq.nlargest(2, range(len(cos_list)), cos_list.__getitem__)

# 获取数值， 输出top 2 得分
fscore = heapq.nlargest(2,cos_list)

# coner case 1: 最高分不匹配问题,阈值0.5
if max(fscore) < 0.5:
    content = ''
    
elif (max(fscore) > 0.6) and (min(fscore) > 0.6):
# coner case 2: 两条文档均匹配问题，阈值0.6
    fnumb1 = fnumb[0]
    fnumb2 = fnumb[1]

    # topk只取1时，且不考虑similarity是否合理，读入文档
    fn1 = fl[fnumb1]
    fn2 = fl[fnumb2]
    file_path1 = f'./Doclib/DocFiles/{fn1}'
    file_path2 = f'./Doclib/DocFiles/{fn2}'
    with open(file_path1, 'r',encoding='utf-8') as f:
        content1 = f.read()
    with open(file_path2, 'r',encoding='utf-8') as f:
        content2 = f.read()
    content = content1 + '\n' + content2

# 正常情况，只取top1
else:
    fn = fl[fnumb[0]]
    file_path = f'./Doclib/DocFiles/{fn}'
    with open(file_path, 'r',encoding='utf-8') as f:
        content = f.read()

In [6]:
fscore

[0.7639769911766052, 0.641689121723175]

In [7]:
type(fscore[0])

float

In [8]:
# GPU 1
# Prompt
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("./GLM6B_model/", trust_remote_code=True)
model = AutoModel.from_pretrained("./GLM6B_model/", trust_remote_code=True).half().cuda()
model = model.eval()

prompt_template = f"""用户不知道文档资料，将文档资料的详细内容转述给用户，以此回答用户问题。
    回答内容不允许超出文档资料内容范围。如果用户的问题与文档资料不相关，请回答“抱歉，我无法回答您的问题。”。另外，答案请使用中文。
    文档资料:
    {content}
    用户问题:
    {question}"""

response, history = model.chat(tokenizer, prompt_template, history=[])
print(response)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

The dtype of attention mask (torch.int64) is not bool


云空间提供多种登录方式：
1.账号+密码登录：输入账号(手机号码/邮箱账号)、密码，勾选同意《隐私政策》点击登录，即可登录云空间。
密码可根据系统提示进行创建。
2.账号+验证码登录：输入您账号绑定的手机号码或邮箱，点击“获取验证码”，图形验证码后手机或邮箱将收到 6 位数的验证码(5 分钟内有效)，填写验证码，点击登录，即可登录云空间。
3.扫码登录：打开手机端云空间app，进入工作台，右上角点击“扫一扫”, 扫描PC/web端的二维码，即可登录云空间。
产品订购成功后，系统会向订购时登记的邮箱内发送云空间订购成功的通知。企业管理员首次使用可通过云空间官网  https://www.ecpan.cn/   在登录页面 “手机号/邮箱号”一栏输入订购时预留的邮箱，点击 【获取验证码】，在邮箱内查看验证码并输入，点击【登录】后，即可进入首页。企业管理员可通过Web端右上角的【管理平台】进入管理平台，在管理平台【通讯录】模块编辑企业部门和人员信息。人员添加完成后，企业成员即可使用云空间。


In [7]:
# 待解决的问题
# 1.GPU显存查询
# 2.GPU分配
# 3.GRPC
# 4.UI界面